In [1]:
## Imports
import pandas as pd
from sqlalchemy import create_engine
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

import nltk
import datasketch
import tqdm
import pytest

In [2]:
df_patient = pd.read_csv("db_clean.csv")

In [3]:
df_patient.head()

,Unnamed: 0,patient_id,given_name,surname,street_number,address_1,address_2,suburb,postcode,state,state_corrected,date_of_birth,dob_date,age,age_inferred,test_year,phone_number,phone_format,phone_geo
0,0,221958,matisse,clarke,13,rene street,westella,ellenbrook,2527,wa,wa,19710708.0,1971-07-08,32.0,32.0,2003.0,08 86018809,True,8.0
1,1,771155,joshua,elrick,23,andrea place,foxdown,east preston,2074,nsw,nsw,19120921.0,1912-09-21,34.0,34.0,1946.0,02 97793152,True,2.0
2,2,231932,alice,conboy,35,mountain circuit,NaN,prospect,2305,nsw,nsw,19810905.0,1981-09-05,22.0,22.0,2003.0,02 20403934,True,2.0
3,3,465838,sienna,craswell,39,cumberlegeicrescent,jodane,henty,3620,wa,wa,19840809.0,1984-08-09,30.0,30.0,2014.0,02 62832318,True,2.0
4,4,359178,joshua,bastiaans,144,lowrie street,NaN,campbell town,4051,nsw,nsw,19340430.0,1934-04-30,31.0,31.0,1965.0,03 69359594,True,3.0


TODO

- [ ] explain what's being done and why
- [ ] add example of fuzzy matching on name_pc on [:1000]

Comme on a fait pour corriger les états, on pourrait directement utiliser un fuzzy matching sur les chaînes de caractères regroupant toutes les informations pertinentes de chaque row (all_info). Cependant, cette opération serait en O(n^2), ce qui n'est pas pratique quand on a 1) une grande base de données et/ou 2) un ordinateur pas vraiment puissant. 

Pour 1000 lignes du dataset, le fuzzy matching réalisé sur une chaîne de caractères contenant nom, prénom et code postal prend un peu plus de 3 minutes sur mon ordinateur. Le réaliser sur l'ensemble du dataset prendrait environ 400 fois plus de temps (en considérant qu'on a 20.000 lignes environ) soit presque 24h, n'imaginons même pas ce que cela donnerait sur les chaînes de caractères bien plus longues comportant toutes les informations.

Au lieu de cela, essayons d'utiliser le locality sensitive hashing, qui regroupe des éléments similaires dans une même classe avec une grande probabilité. Une fois les groupes formés, nous pourrons reprendre le fuzzy matching sur chaque sous-groupe. Qu'il y ait 20 groupes de 1000 ou 2000 groupes de 10 (les groupes ne sont pas forcément de la même taille, c'est une estimation), on arrive à un temps de processing bien plus raisonnable (20 groupes de 1000 : environ 1h (20 \* 3 minutes), 2000 groupes de 10: un peu plus d'une demi-heure (36 minutes, 2000\*3/10.000 car il y aurait 10.000 fois moins d'opérations qu'avec 1000 lignes)).

Cependant, nous risquons de passer à côté de certains doublons, puisqu'il arrive que des chaînes proches ne se retrouvent pas dans le même groupe avec le LSH. C'est pourquoi dans les poids donnés en entrées de LSH, on donne plus de poids au faux négatifs qu'aux faux positifs (qui seront de toute façon filtrés par le fuzzy matching plus tard). On ne peut cependant pas mettre un poids trop fort, en risquant de se retrouver avec deux groupes de 10.000 échantillons chacun par exemple, ce qui en effet minimise le risque de mettre deux éléments se ressemblant dans différents groupes, mais ne nous aide pas pour la phase de fuzzy matching en temps réduit.

Cette méthode de calcul a l'avantage de réduire grandement le temps de calcul par rapport à une méthode plus naïve, ce qui nous permet de gérer une base de données potentiellement de plus en plus grande (si par exemple, s'il y a chaque jour des mises à jour des données avec de nouveaux patients et de nouveaux tests).

**TODO**

- When writing the row, compare values for each columns, majority wins (see how much time it takes)

- Include a "Going further" section:
    - Parmi les dups, on veut garder le patient_id qui est dans les personnes testées ! df_pcr necessary
    - thing about adding external data to check postcode/state accordance
    - adding pcr to know which ones to keep => which ones have pcr results


In [38]:
def better_scorer(s1: str, s2: str) -> int:
    fuzz_ratio = fuzz.ratio(s1, s2)
    fuzz_ratio -= (s2.find(s1)*5)
    return fuzz_ratio

def detect_duplicates(df_patient: pd.DataFrame,
                      interesting_features: list = ["given_name", "surname", "postcode", "phone_number",
                                              "street_number", "address_1", "address_2", "suburb", "postcode", 
                                              "state_corrected", "dob_date", "age"],
                      jaccard_threshold: float=0.4, 
                      num_perm: int=128, 
                      weights: tuple=(0.1,0.9), 
                      nb_ngrams: int=3,
                      fuzzy_threshold: int=65) -> pd.DataFrame:
    
    df_patient["all_info"] = df_patient.apply(lambda row: " ".join(str(info) for info in row[interesting_features]).replace(" nan ", " "), axis=1)

    # Locality sensitive hashing
    lsh = datasketch.MinHashLSH(threshold=jaccard_threshold, num_perm=num_perm, weights=weights)

    minhashes = {}
    for c, i in enumerate(tqdm.tqdm(df_patient["all_info"])):
        minhash = datasketch.MinHash(num_perm=num_perm)
        for d in nltk.ngrams(i, nb_ngrams):
            minhash.update("".join(d).encode('utf-8'))
        lsh.insert(c, minhash)
        minhashes[c] = minhash
        
    df_patient["group_size"] = 0
    for i in tqdm.tqdm(range(len(df_patient))):
        df_patient.at[i, "group_size"] = len(lsh.query(minhashes[i]))
        
    # Find duplicates in groups found
    df_patient["processed"] = False 
    df_patient["original"] = False 
    df_patient["group"] = 0
    group = 0

    # Loop through all rows of dataframe
    for i in tqdm.tqdm(range(len(df_patient))):
        current_element = df_patient.iloc[i]

        # If the current element is not already part of a group
        if not current_element.processed:
            group +=1
            df_patient.at[i, "processed"] = True
            df_patient.at[i, "original"] = True
            df_patient.at[i, "group"] = group

            # Retrieve all elements colliding with the current one
            values_to_compare = df_patient.iloc[lsh.query(minhashes[i])]
            values_to_compare = values_to_compare[values_to_compare.processed == False].all_info
            print(len(values_to_compare))

            # Extract distance of current element to every other colliding element
            result_comparison = process.extract(current_element.all_info, values_to_compare, scorer = better_scorer)
            
            string_match = [comp[0] for comp in result_comparison]
            list_match = [comp[0] for comp in result_comparison if comp[1]>fuzzy_threshold]

            # Mark matches as processed
            for m in list_match:
                df_patient.at[df_patient.loc[df_patient.all_info==m].index, "processed"] = True
                df_patient.at[df_patient.loc[df_patient.all_info==m].index, "group"] = True
                
    return(df_patient)
    
        

In [39]:
import time

start = time.time()
detect_duplicates(df_patient)
end = time.time()

print(end-start)
print(len(df_patient[df_patient.original])/len(df_patient))

  0%|          | 5/19597 [00:00<08:12, 39.81it/s]

198
76
18
87
449
56
81
70
86
199


  0%|          | 20/19597 [00:00<06:36, 49.42it/s]

97
114
66
89
45
7
150
37
215
23
42
60
60
181
66


  0%|          | 35/19597 [00:00<05:51, 55.63it/s]

...

 99%|█████████▉| 19497/19597 [02:03<00:00, 396.25it/s]


0
0
2
1
0
0
2
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
1
1
1
0
0
0
0
0
0
0
0
0
0
1
1
1
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
1
0
0
0

100%|██████████| 19597/19597 [02:03<00:00, 158.66it/s]


0
0
1
0
2
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0


269.83248805999756
0.791294585906006


In [12]:
df_patient.to_csv("australia_covid_no_dup.csv")

**TODO**

Tests

- check that length dedup < length
- write dummy example